In [1]:
import time
import os
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException
import math

In [2]:
def delay(n=5):
    time.sleep(n)

In [3]:
# Url de la Pagina Web
url = 'https://www.linkedin.com/'

# Hacer la solicitud
r = requests.get(url)



In [4]:
# Clase Scrap
class ScrapingSitiosDinamicos:
    def __init__(self):
    
        self.driver = webdriver.Chrome()
        self.perfiles = []
        self.filtro_primario = ""
    
    def delay(self, seconds=3):
        time.sleep(seconds)
        
    def navegar_pagina_web(self):
        url = 'https://www.linkedin.com/'
        self.driver.get(url)
        self.driver.implicitly_wait(10)
    
    # Seleccionando el sector
    def scroll_to_element(self, by, selector):
        try:
            element = WebDriverWait(self.driver, 20).until(
                EC.presence_of_element_located((by, selector))
            )
            self.driver.execute_script("arguments[0].scrollIntoView(true);", element)
            return element
        except (TimeoutException, NoSuchElementException) as e:
            print(f"Element not found: {str(e)}")
            return None

    def iniciar_sesion(self, username, password):
        self.driver.implicitly_wait(10)
        # Localizar y hacer clic en el enlace de iniciar sesión
        login_buttom = self.driver.find_element(By.CSS_SELECTOR, 'a[data-test-id="home-hero-sign-in-cta"]')
        login_buttom.click()
        self.driver.implicitly_wait(10)
        delay()
        # Localizar y ingresar los datos de Login
        username_field = self.driver.find_element(By.ID, 'username')
        password_fiel = self.driver.find_element(By.ID, 'password')
        submit_buttom = self.driver.find_element(By.CLASS_NAME, 'login__form_action_container ')
        # Introducir las credenciales
        username_field.send_keys(username)
        password_fiel.send_keys(password)
        submit_buttom.click()
        delay()

    def buscar_perfil(self, busqueda):
        self.busqueda = busqueda
        self.driver.implicitly_wait(20)  # Aumentar tiempo de espera implícito
        
        try:
            print("Buscando el campo de búsqueda")
            # Buscar campo de búsqueda
            search_field = WebDriverWait(self.driver, 20).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[placeholder="Buscar"]'))
                )
            search_field.click()
            search_field.send_keys(busqueda)
            search_field.send_keys(Keys.RETURN)
            
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            
    def aplicar_filtros(self, filtro_primario: str, ubicacion: str, sector = None, categorias = None):
        try:
            print("Aplicando filtros")
            # Filtros
            try:
                if filtro_primario == 'Personas':
                    # Localizar boton de filtro "Personas"
                    buttom_person_filter = WebDriverWait(self.driver, 20).until(
                        EC.element_to_be_clickable((By.XPATH, '//ul/li[contains(@class, "search-reusables__primary-filter")]/button[text()[contains(., "Personas")]]'))
                    )
                    
                    if buttom_person_filter.text.strip() == 'Personas':
                        # Mensaje Debug 
                        print(f'Se ha localizado en el elemento {buttom_person_filter.text}')
                        text = buttom_person_filter.text
                        buttom_person_filter.click()
                        
                elif filtro_primario == 'Empresas':
                    # Localizar boton de filtro "Empresas"
                    buttom_person_filter = WebDriverWait(self.driver, 20).until(
                        EC.element_to_be_clickable((By.XPATH, '//ul/li[contains(@class, "search-reusables__primary-filter")]/button[text()[contains(., "Empresas")]]'))
                    )
                    
                    if buttom_person_filter.text.strip() == 'Empresas':
                        # Mensaje Debug 
                        print(f'Se ha localizado en el elemento {buttom_person_filter.text}')
                        text = buttom_person_filter.text
                        buttom_person_filter.click()
                
            except Exception as e:
                print(f'Ha ocurrido un problema con la seleccion de elementos {e}')
                
            # Mensaje Debug
            print(f'Se ha hecho clic en el elemento con el texto: {text}')

            # Seleccionando el boton de Todos los filtros
            buttom_all_filter = WebDriverWait(self.driver, 20).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-expanded="false"][aria-label="Mostrar todos los filtros. Al hacer clic en este botón, se muestran todas las opciones de filtros disponibles."]'))
            )
            print(f'Se ha localizado el elemento {buttom_all_filter.text}')
            
            buttom_all_filter.click()
            
            print(f'Se ha hecho clic en el elemento {buttom_all_filter.text}')
            
            self.delay()
            
            if ubicacion != None:
            
                empresa_selector = '/html/body/div[3]/div/div/div[2]/ul/li[5]/fieldset/legend'
                self.scroll_to_element(By.XPATH, empresa_selector)

                self.delay()
                
                # Seleccionando Añadir Ubicacion y enviamos el key
                button_location_filter = WebDriverWait(self.driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, '//div[@class="search-reusables__secondary-filters-values"]//ul//li[contains(@class, "search-reusables__filter-value-item") and contains(@class, "mt4")]/button[contains(@class, "reusable-search-filters-advanced-filters__add-filter-button") and .//span[text()="Añade una ubicación"]]'))
                )
                button_location_filter.click()
                
                self.delay()
            
                input_location = WebDriverWait(self.driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, '//input[@placeholder="Añade una ubicación"]'))
                )
                input_location.send_keys(ubicacion)
                
                self.delay()
                    
                
                # Verificar si el elemento está presente primero
                result_location = WebDriverWait(self.driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "basic-typeahead__triggered-content") and contains(@id, "triggered-expanded")]'))
                )
                
                # Ahora verificar si el elemento es clickeable
                result_location = WebDriverWait(self.driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, '//div[contains(@class, "basic-typeahead__triggered-content") and contains(@id, "triggered-expanded")]'))
                )
                result_location.click()
                print("Se ha hecho clic en el elemento")
                
                self.delay()
                
            if sector != None:
            
                sector_selector = '/html/body/div[3]/div/div/div[2]/ul/li[8]/fieldset/legend'
                scroll_sector_filter = self.scroll_to_element(By.XPATH, sector_selector) 
                print(f'Se ha localizado el elemento {scroll_sector_filter.text}')      
                
                ''' Funcion para Sector '''
                
                # Seleccionando Añadir Sector y enviamos el key
                button_sector_filter = WebDriverWait(self.driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, '//span[text()="Añade un sector"]'))
                )
                button_sector_filter.click()
                
                self.delay()
                
                input_sector = WebDriverWait(self.driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, '//input[@placeholder="Añade un sector"]'))
                )
                input_sector.send_keys(sector)
                
                self.delay()
                    
                
                # Verificar si el elemento está presente primero
                result_sector = WebDriverWait(self.driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "basic-typeahead__triggered-content") and contains(@id, "triggered-expanded")]'))
                )
                
                # Ahora verificar si el elemento es clickeable
                result_sector = WebDriverWait(self.driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, '//div[contains(@class, "basic-typeahead__triggered-content") and contains(@id, "triggered-expanded")]'))
                )
                result_sector.click()
                print("Se ha hecho clic en el elemento")
                
                self.delay()
                
            if categorias != None:
            
                categoria_selector = '/html/body/div[3]/div/div/div[2]/ul/li[11]/fieldset/legend'
                scroll_categoria_filter = self.scroll_to_element(By.XPATH, categoria_selector)
                print(f'Se ha localizado el elemento {scroll_categoria_filter.text}')

                ''' Funcion para Categorías de servicios '''
                # Obtener todos los elementos de categorías
                categorias_procesadas = set()
        
                try:
                    # Obtener todos los elementos de categorías
                    categorias_elements = WebDriverWait(self.driver, 20).until(
                        EC.presence_of_all_elements_located((By.XPATH, '//li[@class="search-reusables__filter-value-item"]/label[@class="search-reusables__value-label"]/p[@class="display-flex"]/span[@class="t-14 t-black--light t-normal"]'))
                    )
                    count_categorias = len(categorias_elements)
                    print(f"Se encontraron {count_categorias} categorías.")
                except TimeoutException:
                    print("TimeoutException: No se encontraron categorías.")
                    return

                for i in range(count_categorias):
                    try:
                        button_categorias_filter = categorias_elements[i]
                        text_other = button_categorias_filter.text.strip()

                        if text_other in categorias and text_other not in categorias_procesadas:
                            button_categorias_filter.click()
                            categorias_procesadas.add(text_other)
                    
                    except TimeoutException:
                        print(f"TimeoutException: No se pudo hacer clic en el botón {i}.")
                        continue
                    except ElementClickInterceptedException:
                        print(f"ElementClickInterceptedException: El clic fue interceptado por otro elemento para el botón {i}.")
                        continue
                    except NoSuchElementException:
                        print(f"NoSuchElementException: El botón {i} no se encontró.")
                        continue
                    except Exception as e:
                        print(f"Exception: Ocurrió un error con el botón {i}: {e}")
                        continue
                    
                for categoria in categorias:
                    if categoria not in categorias_procesadas:
                        try:
                            add_service_button = WebDriverWait(self.driver, 20).until(
                                EC.element_to_be_clickable((By.XPATH, '//span[text()="Añade un servicio"]'))
                            )
                            add_service_button.click()
                            self.delay()
                            
                            input_categorias = WebDriverWait(self.driver, 20).until(
                                EC.element_to_be_clickable((By.XPATH, '//input[@placeholder="Añade un servicio"]'))
                            )
                            input_categorias.clear()  # Borrar el contenido del campo de entrada antes de enviar nuevas teclas
                            input_categorias.send_keys(categoria)
                            self.delay()

                            result_categorias = WebDriverWait(self.driver, 20).until(
                                EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "basic-typeahead__triggered-content") and contains(@id, "triggered-expanded")]'))
                            )
                            print(f"Elemento encontrado para la categoría: {categoria}")

                            found = False
                            for result in result_categorias:
                                if categoria.lower() in result.text.lower().strip():
                                    result.click()
                                    print(f"Se ha hecho clic en el elemento para la categoría: {categoria}")
                                    found = True
                                    categorias_procesadas.add(categoria)
                                    break
                            
                            if not found:
                                print(f"No se encontró coincidencia exacta para la categoría: {categoria}")
                            
                        except TimeoutException:
                            print(f"TimeoutException: No se encontraron resultados para la categoría: {categoria}")
                        except ElementClickInterceptedException:
                            print(f"ElementClickInterceptedException: El clic fue interceptado por otro elemento para la categoría: {categoria}")
                        except NoSuchElementException:
                            print(f"NoSuchElementException: El elemento no se encontró en la página para la categoría: {categoria}")
                        except Exception as e:
                            print(f"Exception: Ocurrió un error con la categoría {categoria}: {e}")
                
            # Seleccionando el boton para mostrar los resultados
            button_show_result_filter = WebDriverWait(self.driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div/div/div[3]/div/button[2]'))
            )
            print(f'Se ha localizado el elemento {button_show_result_filter.text}') 
            text_1 = button_show_result_filter.text
            button_show_result_filter.click()   
            print(f'Se ha hecho clic en el elemento {text_1}')
            
        except Exception as e:
                print(f"An error occurred: {str(e)}")

    def next_page(self):
        next_button_selectors = '//button[@aria-label="Siguiente"]'
        
        # Scroll hasta el final de la página
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # Scroll hasta el final de pagina
        print('Se ha hecho Scroll hasta el final de la pagina')
        
        try:
            next_button = WebDriverWait(self.driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, next_button_selectors))
            )
            
            if next_button:
                text_next_nage = next_button.text
                next_button.click()
                print(f'Se ha hecho click en el elemento {text_next_nage}')

        except TimeoutException:
            print(f"TimeoutException: No se encontró el botón con el selector {next_button_selectors}")
            
        except NoSuchElementException:
            print(f"NoSuchElementException: No se encontró el botón con el selector {next_button_selectors}")
    
    def extraer_perfil(self):
        # Localizar el elemento que contiene el número de resultados usando XPATH
        count_result_element = self.driver.find_element(By.XPATH, '//div[@class="search-marvel-srp"]//div[@class="search-results-container"]//h2').text.strip()
        count_result = self._extract_number_from_string(count_result_element)
        print(f'Se ha encontrado un total de {count_result} resultados')
        
        count_next_page = math.ceil(count_result / 10) # # Contabilizar la cantidad de veces que hay que pasar página
        
        self.extraer_info(count_next_page)
        
    # Mover el mouse a un elemento específico
    def abrir_vinculo_en_nueva_pestana_y_cambiar(self, by, selector):
        try:
            link_element = WebDriverWait(self.driver, 30).until(
                EC.element_to_be_clickable((by, selector))
            )
            actions = ActionChains(self.driver)
            actions.key_down(Keys.CONTROL).click(link_element).key_up(Keys.CONTROL).perform()
            print("El vínculo se ha abierto en una nueva pestaña")

            new_tab = self.driver.window_handles[-1]
            self.driver.switch_to.window(new_tab)
            print("Se ha cambiado el foco a la nueva pestaña")
        except Exception as e:
            print(f"No se pudo abrir el vínculo en una nueva pestaña: {str(e)}")
    
    def cerrar_pestana_actual_y_volver(self):
        try:
            # Cerrar la pestaña actual
            self.driver.close()

            # Cambiar el foco de vuelta a la primera pestaña
            original_tab = self.driver.window_handles[0]
            self.driver.switch_to.window(original_tab)
            print("Se ha vuelto a la pestaña original")
        except Exception as e:
            print(f"No se pudo cerrar la pestaña actual: {str(e)}")
        
    def extraer_info(self, count_next_page):
        try:
            self.perfiles_encontrados = []

            for page in range(1, count_next_page + 1):
                # Esperar a que la lista de perfiles esté presente
                perfiles_element = WebDriverWait(self.driver, 20).until(
                    EC.presence_of_all_elements_located((By.XPATH, '//ul[@class="reusable-search__entity-result-list list-style-none"]//li'))
                )
                
                # Iterar sobre la lista de elementos li
                for index, perfil in enumerate(perfiles_element, start=1):
                    try:
                        print(f"Procesando perfil {index}")
                        
                        # Hacer Scroll al perfil actual
                        self.driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'nearest'});", perfil)

                        # Encontrar el enlace del perfil y obtener la URL
                        perfil_link = perfil.find_element(By.XPATH, './/a[contains(@class, "app-aware-link") and contains(@href, "linkedin.com/company/")]')
                        perfil_url = perfil_link.get_attribute("href")
                        
                        # Abrir la URL del perfil en una nueva pestaña
                        self.abrir_vinculo_en_nueva_pestana_y_cambiar(By.XPATH, f'.//a[@href="{perfil_url}"]')
                        
                        # Inicializar variables para la información a extraer
                        name_text = None
                        telefono_text = None
                        sector_text = None
                        
                        try:
                            # Extraer nombre
                            name_element = WebDriverWait(self.driver, 20).until(
                                EC.presence_of_element_located((By.XPATH, '//div[@class="block mt2"]//div//h1[@title]'))
                            )
                            name_text = name_element.text.strip()
                            print(f"Nombre: {name_text}")

                            # Probar diferentes formas de seleccionar el elemento "Acerca de"
                            acerca_de_element = WebDriverWait(self.driver, 30).until(
                                EC.element_to_be_clickable((By.XPATH, '//a[contains(@href, "/about")]'))
                            )
                            acerca_de_element.click()

                            self.delay()

                            # Extraer teléfono
                            try:
                                telefono_element = WebDriverWait(self.driver, 10).until(
                                    EC.presence_of_element_located((By.XPATH, '//dt[@class="mb1 text-heading-medium"]//h3[contains(., "Teléfono")]'))
                                )
                                if telefono_element:
                                    self.scroll_to_element(By.XPATH, '//dt[@class="mb1 text-heading-medium"]//h3[contains(., "Teléfono")]')
                                    telefono_text = self.driver.find_element(By.XPATH, '//a[@href[contains(., "tel:")]]/span[@aria-hidden="true" and contains(@class, "link-without-visited-state")]').text.strip()
                                    print(f"Teléfono: {telefono_text}")
                            except:
                                print(f"Teléfono no encontrado para el perfil {index}")

                            self.delay()

                            # Extraer sector
                            try:
                                sector_element = WebDriverWait(self.driver, 10).until(
                                    EC.presence_of_element_located((By.XPATH, '//dt[@class="mb1"]/h3[contains(., "Sector")]'))
                                )
                                if sector_element:
                                    self.scroll_to_element(By.XPATH, '//dt[@class="mb1"]/h3[contains(., "Sector")]')
                                    sector_text = self.driver.find_element(By.XPATH, '//dt[h3[text()="Sector"]]/following-sibling::dd[contains(@class, "mb4") and contains(@class, "t-black--light") and contains(@class, "text-body-medium")]').text.strip()
                                    print(f"Sector: {sector_text}")
                            except:
                                print(f"Sector no encontrado para el perfil {index}")

                            # Guardar los datos extraídos
                            self.perfiles_encontrados.append({
                                'Nombre': name_text,
                                'Teléfono': telefono_text,
                                'Sector': sector_text
                            })

                            # Cerrar la pestaña actual y volver a la original
                            self.cerrar_pestana_actual_y_volver()
                            
                        except Exception as e:
                            print(f"Error al extraer información del perfil {index}: {str(e)}")
                            self.cerrar_pestana_actual_y_volver()
                            
                    except Exception as e:
                        print(f"Error al procesar el perfil {index}: {str(e)}")
                        
                if page < count_next_page:
                    self.next_page()
                    # Agregar un pequeño retraso para evitar sobrecargar el servidor
                    self.delay(2)
                        
            self.guardar_perfiles()
            
        except Exception as e:
            print(f'Ha ocurrido un problema a la hora de extraer información: {e}')

            
        
    def guardar_perfiles(self):
        # Guardar todos los perfiles en un archivo Excel después de extraer la información de todas las páginas
        df = pd.DataFrame(self.perfiles_encontrados)
        name_file = f'Perfiles_{self.busqueda}.xlsx'
        df.to_excel(name_file, index=False)
        print('Se ha exportado con exito el archivo')
    
    def _extract_number_from_string(self, text):
        import re
        match = re.search(r'\d+', text.replace(',', ''))
        if match:
            return int(match.group())
        raise ValueError(f"Couldn't find a number in the text: {text}")
    
    



In [5]:
try:
    print("Iniciando scraping")
    scraping = ScrapingSitiosDinamicos()
    scraping.navegar_pagina_web()
    scraping.iniciar_sesion('melphisdejesusp@gmail.com', 'Fam@Melphis.20.Alan.21.Alisone.16.Megan.19')
    scraping.buscar_perfil('Logistica')
    scraping.filtro_primario = 'Empresas'
    scraping.aplicar_filtros(scraping.filtro_primario, "República Dominicana")
    
    count_categorias = 5  # Número de categorías en tu página
    
    scraping.extraer_perfil()
    
    
    
except KeyboardInterrupt as e:
    print(f'Ha ocurrido un error, se ha interrumpido el proceso {str(e)}')
    scraping.driver.quit()



Iniciando scraping
Buscando el campo de búsqueda
Aplicando filtros
Se ha localizado en el elemento Empresas
Se ha hecho clic en el elemento con el texto: Empresas
Se ha localizado el elemento Todos los filtros
Se ha hecho clic en el elemento Todos los filtros
Se ha hecho clic en el elemento
Se ha localizado el elemento Mostrar resultados
Se ha hecho clic en el elemento Mostrar resultados
Se ha encontrado un total de 198 resultados
Procesando perfil 1
El vínculo se ha abierto en una nueva pestaña
Se ha cambiado el foco a la nueva pestaña
Nombre: Logística Legal | Abogados Estratégicos
Teléfono: 1-809-518-8765
Sector: Servicios jurídicos
Se ha vuelto a la pestaña original
Procesando perfil 2
El vínculo se ha abierto en una nueva pestaña
Se ha cambiado el foco a la nueva pestaña
Nombre: Observatorio Nacional de Logística y Transporte de Carga
Teléfono no encontrado para el perfil 2
Sector: Servicios de investigación
Se ha vuelto a la pestaña original
Procesando perfil 3
Error al procesar 